In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
import sys

print("TensorFlow version : " + tf.__version__)
print("Python version : " + sys.version)

TensorFlow version : 2.9.2
Python version : 3.8.16 (default, Dec  7 2022, 01:12:13) 
[GCC 7.5.0]


In [ ]:
import os

os.environ['KAGGLE_CONFIG_DIR'] = "drive/MyDrive/kaggle/"

!kaggle datasets download -d jessicali9530/celeba-dataset
!unzip -q celeba-dataset.zip -d .

 98% 1.30G/1.33G [00:15<00:00, 243MB/s]
100% 1.33G/1.33G [00:15<00:00, 91.1MB/s]


In [ ]:
from PIL import Image
import os
import numpy as np

file_list = os.listdir('/content/img_align_celeba/img_align_celeba')
images = []

for i in file_list[0:50000]:
  images_to_number = Image.open('/content/img_align_celeba/img_align_celeba/' + i).crop((30,40,160,170)).convert('L').resize(( 64,64))
  images.append(np.array(images_to_number))

import matplotlib.pyplot as plt
plt.imshow(images[1])
plt.show()

images = np.array(images)
print(images.shape)



In [ ]:
images = np.divide(images,255)
images = images.reshape(50000,64,64,1)
print(images.shape)

(50000, 64, 64, 1)


In [ ]:
discriminator = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), strides = (2, 2), padding = 'same', input_shape=[64, 64, 1]),
    tf.keras.layers.LeakyReLU(alpha = 0.2),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Conv2D(64, (3, 3), strides = (2, 2), padding = 'same'),
    tf.keras.layers.LeakyReLU(alpha = 0.2),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Conv2D(64, (3, 3), strides = (2, 2), padding = 'same'),
    tf.keras.layers.LeakyReLU(alpha = 0.2),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Conv2D(64, (3, 3), strides = (2, 2), padding = 'same'),
    tf.keras.layers.LeakyReLU(alpha = 0.2),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

In [ ]:
generator = tf.keras.models.Sequential([
  tf.keras.layers.Dense(4 * 4 * 256, input_shape = (100,)),
  tf.keras.layers.Reshape((4, 4, 256)),
  tf.keras.layers.Conv2DTranspose(256, 3, strides=2, padding='same'),
  tf.keras.layers.LeakyReLU(alpha=0.2),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Conv2DTranspose(128, 3, strides=2, padding='same'),
  tf.keras.layers.LeakyReLU(alpha=0.2),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Conv2DTranspose(64, 3, strides=2, padding='same'),
  tf.keras.layers.LeakyReLU(alpha=0.2),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Conv2DTranspose(1, 3, strides=2, padding='same', activation='tanh')
])

In [ ]:
GAN = tf.keras.models.Sequential([generator, discriminator])

discriminator.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy'
)
discriminator.trainable = False

GAN.compile(optimizer = 'adam', loss = 'binary_crossentropy')

In [ ]:
def picture():
  random_number = np.random.uniform(-1, 1, size = (10, 100))
  predict = generator.predict(random_number)

  for i in range(10):
    plt.subplot(2, 5, i + 1)
    plt.imshow(predict[i].reshape(64, 64), cmap = 'gray')
    plt.axis('off')
  plt.tight_layout()
  plt.show()

In [ ]:
for j in range(300):
  print(f'{j} epoch 입니다.')
  picture()

  for i in range(50000//128):
    
    #Discriminator
    real_image = images[i * 128 : (i + 1) * 128]
    matrix_filled_with_ones = np.ones(shape = (128, 1))
    loss1 = discriminator.train_on_batch(real_image, matrix_filled_with_ones)

    random_number = np.random.uniform(-1, 1, size = (128, 100))
    face_images = generator.predict(random_number)
    matrix_filled_with_zeros = np.zeros(shape = (128, 1))
    loss2 = discriminator.train_on_batch(face_images, matrix_filled_with_zeros)

    #Generator

    random_number = np.random.uniform(-1, 1, size = (128, 100))
    matrix_filled_with_ones = np.ones(shape = (128,1))

    loss3 = GAN.train_on_batch(random_number, matrix_filled_with_ones)

  print(f'Discriminator의 loss는 {loss1 + loss2}입니다.\nGenerator의 loss는 {loss3}입니다.')

In [ ]:
GAN.save('face_generator.h5')